# 레이블 명칭 추가 및 데이터 인코딩 변경
- 인코딩된 레이블 디코딩
- 윈도우에서 한글이 깨지지 않도록 변경

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./resources/raw_data/train.csv", index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: './resources/raw_data/train.csv'

In [3]:
df

,text,target,target_name
ID,,,
ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,IT과학
ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,사회
ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,정치
ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,경제
ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,세계
...,...,...,...
ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,세계
ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,정치
ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4,IT과학


In [13]:
# label_dic = {0:"정치", 1:"경제", 2:"사회", 3:"생활문화", 4:"세계", 5:"IT과학", 6:"스포츠"}
label_encodeing_dic = {'생활문화': 0, '스포츠': 1, '정치': 2, '사회': 3, 'IT과학': 4, '경제': 5, '세계': 6}
label_decoding_dic = {v:k for k, v in label_encodeing_dic.items()}

In [14]:
df['target_name'] = [label_decoding_dic[x] for x in df['target']]

In [15]:
df.to_csv("./resources/raw_data/train.csv", encoding='utf-8-sig', index=0)

In [16]:
df

,ID,text,target,target_name
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,IT과학
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,사회
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,정치
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,경제
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,세계
...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,세계
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,정치
2797,ynat-v1_train_02797,텔레그램+한D 등h亞서 2시간H다운…C버T정gf39종!2보,4,IT과학
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,스포츠


# 랜덤 노이즈 제거
데이터는 (1) 레이블이 잘못되어 있는 경우 1,400, (2) 랜덤 노이즈가 있는 경우 1,400, (3) 정상 200 케이스로 분류 가능

Cleanlab 이든 레이블이 잘못된 경우를 찾기 위해서는 정상 데이터의 비율을 늘려야함

따라서, 랜덤 노이즈를 정상 케이스로 돌리는 작업이 먼저 필요

## Translation to huggingface model
- 랜덤 노이즈를 반영해서 잘 번역하지 못하는 것 같음.
- 구글 Translation 같은 좀 더 강한한 모델 찾아야할 듯

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

repo = "davidkim205/iris-7b"
model = AutoModelForCausalLM.from_pretrained(repo, torch_dtype=torch.bfloat16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(repo)

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [20]:
df['token_len'] = [len(tokenizer(x)['input_ids']) for x in df['text']]

In [26]:
# 아래 결과에 따라 max new token은 128로 설정
df['token_len'].describe()

count    2800.000000
mean       31.678214
std         6.457354
min         9.000000
25%        28.000000
50%        32.000000
75%        36.000000
max        53.000000
Name: token_len, dtype: float64

In [27]:
def generate(prompt):
    encoding = tokenizer(
        prompt,
        return_tensors='pt',
        return_token_type_ids=False
    ).to("cuda")
    gen_tokens = model.generate(
        **encoding,
        max_new_tokens=128,
        temperature=1.0,
        num_beams=5,
    )
    prompt_end_size = encoding.input_ids.shape[1]
    result = tokenizer.decode(gen_tokens[0, prompt_end_size:])

    return result


def translate_ko2en(text):
    prompt = f"[INST] 다음 문장을 영어로 번역하세요.{text} [/INST]"
    return generate(prompt)


def translate_en2ko(text):
    prompt = f"[INST] 다음 문장을 한글로 번역하세요.{text} [/INST]"
    return generate(prompt)

In [30]:
en_text_lst = []
ko_text_lst = []
for i, row in df.iterrows():
    text = row['text']
    print('or_text ', text)
    en_text = translate_ko2en(text)
    en_text_lst.append(en_text)
    ko_text = translate_en2ko(en_text)
    ko_text_lst.append(ko_text_lst)
    print('en_text', en_text)
    print('ko_text', ko_text)
    if i == 10:
        break

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


or_text  정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


en_text JEONGI : FA1 MISSAZ KT (USE PERIOD 2E DAN) QDISTINCT U2BO</s>
ko_text JEONGI : FA1 MISSAZ KT (USE PERIOD 2E DAN) QDISTINCT U2BO</s>
or_text  K찰.국DLwo 로L3한N% 회장 2 T0&}송=


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

en_text How much did President Bid surprise you...comprehensive visit to the Gyeongju earthquake site</s>
ko_text 비드 대통령이 경주 지진 현장을 포괄적으로 방문한 것이 얼마나 놀라웠나요?</s>
or_text  듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


en_text I'm looking forward to the release of the dual-sim iPhone in the second half of the year...</s>
ko_text 저는 올 하반기 듀얼 심 아이폰의 출시를 기대하고 있습니다...</s>
or_text  oi 매력 R모h츠a열#w3약 >l·주가 고Q/진


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


en_text WOI Attractive RMohwtsa열#w3약 >l·주가 고Q/진</s>
ko_text WOI Attractive RMohwtsa열#w3약 >l·주가 고Q/진</s>


## LLM 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [5]:
from huggingface_hub import login

login()

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [15]:
EXAONE_FEW_SHOT_TEMP = '''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
{}

예시 1:
원본: NH투자 1월 옵션 만기일 매도 우세
교정: NH투자 1월 옵션 만기일 매도 우세

예시 2:
원본: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
교정: 정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)

예시 3:
원본: K찰.국DLwo 로L3한N% 회장 2 T0&송=
교정: 경찰, '국회 불법 로비' 한어총 회장 등 20명 송치

예시 4:
원본: m 김정) 자주통일 새,?r열1나가야1보
교정: 김정은 "자주통일 새시대 열어나가야"(2보)

예시 5:
원본: pI美대선I앞두고 R2fr단 발] $비해 감시 강화
교정: 軍 "美대선 앞두고 北 무수단 발사 대비해 감시 강화"

예시 6:
원본: 프로야구~롯TKIAs광주 경기 y천취소
교정: 프로야구 롯데-KIA 광주 경기 우천취소

[|assistant|]'''

In [18]:
test_text = "정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보"
test_prompt = EXAONE_FEW_SHOT_TEMP.format(test_text)
test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.\n정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보\n\n예시 1:\n원본: NH투자 1월 옵션 만기일 매도 우세\n교정: NH투자 1월 옵션 만기일 매도 우세\n\n예시 2:\n원본: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보\n교정: 정부, \'주파수 미사용\' KT에 이용기간 2년 단축 처분(종합2보)\n\n예시 3:\n원본: K찰.국DLwo 로L3한N% 회장 2 T0&송=\n교정: 경찰, \'국회 불법 로비\' 한어총 회장 등 20명 송치\n\n예시 4:\n원본: m 김정) 자주통일 새,?r열1나가야1보\n교정: 김정은 "자주통일 새시대 열어나가야"(2보)\n\n예시 5:\n원본: pI美대선I앞두고 R2fr단 발] $비해 감시 강화\n교정: 軍 "美대선 앞두고 北 무수단 발사 대비해 감시 강화"\n\n예시 6:\n원본: 프로야구~롯TKIAs광주 경기 y천취소\n교정: 프로야구 롯데-KIA 광주 경기 우천취소\n\n[|assistant|]'

In [20]:
input_ids = tokenizer(test_prompt, return_tensors="pt")['input_ids']

output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=32
)
print(tokenizer.decode(output[0]))

[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보

예시 1:
원본: NH투자 1월 옵션 만기일 매도 우세
교정: NH투자 1월 옵션 만기일 매도 우세

예시 2:
원본: 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
교정: 정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)

예시 3:
원본: K찰.국DLwo 로L3한N% 회장 2 T0&송=
교정: 경찰, '국회 불법 로비' 한어총 회장 등 20명 송치

예시 4:
원본: m 김정) 자주통일 새,?r열1나가야1보
교정: 김정은 "자주통일 새시대 열어나가야"(2보)

예시 5:
원본: pI美대선I앞두고 R2fr단 발] $비해 감시 강화
교정: 軍 "美대선 앞두고 北 무수단 발사 대비해 감시 강화"

예시 6:
원본: 프로야구~롯TKIAs광주 경기 y천취소
교정: 프로야구 롯데-KIA 광주 경기 우천취소

[|assistant|]교정: 정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)[|endofturn|]


In [21]:
# EXAONE_FEW_SHOT_TEMP = '''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
# [|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
# {}
# [|assistant|]{}[|endofturn|]'''

In [22]:
# few_text = ["NH투자 1월 옵션 만기일 매도 우세", "정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보", "K찰.국DLwo 로L3한N% 회장 2 T0&}송="
#             "m 김정) 자주통일 새,?r열1나가야1보", "pI美대선I앞두고 R2fr단 발] $비해 감시 강화", "프로야구~롯TKIAs광주 경기 y천취소"]
# few_retext = ["NH투자 1월 옵션 만기일 매도 우세", "정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)","경찰, '국회 불법 로비' 한어총 회장 등 20명 송치"
#               "김정은 \"자주통일 새시대 열어나가야\"(2보)", "軍 \"美대선 앞두고 北 무수단 발사 대비해 감시 강화\"", "프로야구 롯데-KIA 광주 경기 우천취소"]

In [23]:
# few_shop_promt = ''
# for text, topic in zip(few_text, few_retext):
#     few_shop_promt = few_shop_promt + EXAONE_FEW_SHOT_TEMP.format(text, topic)

In [24]:
# few_shop_promt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.\nNH투자 1월 옵션 만기일 매도 우세\n[|assistant|]NH투자 1월 옵션 만기일 매도 우세[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.\n정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보\n[|assistant|]정부, \'주파수 미사용\' KT에 이용기간 2년 단축 처분(종합2보)[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.\nK찰.국DLwo 로L3한N% 회장 2 T0&}송=m 김정) 자주통일 새,?r열1나가야1보\n[|assistant|]경찰, \'국회 불법 로비\' 한어총 회장 등 20명 송치김정은 "자주통일 새시대 열어나가야"(2보)[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.\npI美대선I앞두고 R2fr단 발] $비해 감시 강화\n[|assistant|]軍 "美대선 앞두고 北 무수단 발사 대비해 감시 강화"[|endo

In [25]:
# EXAONE_TEMP = '''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
# [|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
# {}
# [|assistant|]'''

In [26]:
# test_text = "정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보"
# test_prompt = EXAONE_TEMP.format(test_text)
# test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.\n정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보\n[|assistant|]'

In [27]:
# input_ids = tokenizer(few_shop_promt + test_prompt, return_tensors="pt")['input_ids']

# output = model.generate(
#     input_ids.to("cuda"),
#     eos_token_id=tokenizer.eos_token_id,
#     max_new_tokens=128
# )
# print(tokenizer.decode(output[0]))

[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
NH투자 1월 옵션 만기일 매도 우세
[|assistant|]NH투자 1월 옵션 만기일 매도 우세[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보
[|assistant|]정부, '주파수 미사용' KT에 이용기간 2년 단축 처분(종합2보)[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
K찰.국DLwo 로L3한N% 회장 2 T0&}송=m 김정) 자주통일 새,?r열1나가야1보
[|assistant|]경찰, '국회 불법 로비' 한어총 회장 등 20명 송치김정은 "자주통일 새시대 열어나가야"(2보)[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 맞춤법을 교정해줘. 답변은 간결하고 단답으로 해. 설명은 절대 하지마.
pI美대선I앞두고 R2fr단 발] $비해 감시 강화
[|assistant|]軍 "美대선 앞두고 北 무수단 발사 대비해 감시 강화"[|endofturn|][|system|]

# 레이블 수정

### LLM 기반 레이블 수정

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [4]:
from huggingface_hub import login

login()

Token has not been saved to git credential helper.


In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct")

In [8]:
# EXAONE_FEW_SHOT_TEMP = '''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
# [|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 간결하고 단답으로 해.
# {}

# 예시 1:
# 헤드라인: 황총리 각 부처 비상대비태세 철저히 강구해야
# 토픽: 정치

# 예시 2:
# 헤드라인: 금융시장 충격 일단 소강국면…주가 낙폭 줄고 환율도 하락
# 토픽: 경제

# 예시 3:
# 헤드라인: 월미도 새 모노레일 내년 추석엔 달릴 수 있을까
# 토픽: 사회

# 예시 4:
# 헤드라인: 벚꽃 와인 마시며 봄 즐기세요
# 토픽: 생활문화

# 예시 5:
# 헤드라인: 트럼프 한국 등 방위비분담금 더 내라…양방향 도로 돼야
# 토픽: 세계

# 예시 6:
# 헤드라인: 네이버 모바일 연예판에도 AI 콘텐츠 추천 시스템 적용
# 토픽: IT과학

# 예시 7:
# 헤드라인: 대한항공 우리카드 꺾고 3연승…GS칼텍스 1라운드 전승종합
# 토픽: 스포츠

# 예시 8:
# 헤드라인: 中공안 장쑤 화학공단 폭발참사 책임자들 구금
# 토픽: 사회

# 예시 9:
# 헤드라인: 고3 확진자 나왔는데…부산서도 조마조마 3차 등교수업종합
# 토픽: 사회

# 예시 10:
# 헤드라인: 제주 4·3 아픔 보듬는 시집…꽃보다 먼저 다녀간 이름들
# 토픽: 사회

# [|assistant|]'''


In [48]:
EXAONE_FEW_SHOT_TEMP = '''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠 중에 있어. 가능한 모든 주제를 최대 3개까지 확률이 높은 순으로 나열해서 줘. 
주제를 선택한 이유를 키워드로 답변해. 설명은 간결하게 해.
### 분석포맷
1.주제: 정치 -> 키워드: "이유"
2.주제: 경제 -> 키워드: "이유"
3.주제: 사회 -> 키워드: "이유"

{}
[|assistant|]'''

In [59]:
test_text = '월드컵 천재 사령탑 시코르 감독 한국전 승리...'
test_prompt = EXAONE_FEW_SHOT_TEMP.format(test_text)
test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠 중에 있어. 가능한 모든 주제를 최대 3개까지 확률이 높은 순으로 나열해서 줘. \n주제를 선택한 이유를 키워드로 답변해. 설명은 간결하게 해.\n### 분석포맷\n1.주제: 정치 -> 키워드: "이유"\n2.주제: 경제 -> 키워드: "이유"\n3.주제: 사회 -> 키워드: "이유"\n\n월드컵 천재 사령탑 시코르 감독 한국전 승리...\n[|assistant|]'

In [60]:
input_ids = tokenizer(test_prompt, return_tensors="pt")['input_ids']
output = model.generate(
    input_ids.to("cuda"),
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128
)
print(tokenizer.decode(output[0]))

[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠 중에 있어. 가능한 모든 주제를 최대 3개까지 확률이 높은 순으로 나열해서 줘. 
주제를 선택한 이유를 키워드로 답변해. 설명은 간결하게 해.
### 분석포맷
1.주제: 정치 -> 키워드: "이유"
2.주제: 경제 -> 키워드: "이유"
3.주제: 사회 -> 키워드: "이유"

월드컵 천재 사령탑 시코르 감독 한국전 승리...
[|assistant|]1. 주제: 스포츠 -> 키워드: "시시코르", "감독", "승리"
2. 주제: 세계 -> 키워드: "월드컵", "한국전"
3. 주제: 생활문화 -> 키워드: "천재", "감독"

설명:
1. 스포츠: 시시코르 감독과 관련된 내용이며, 승리를 언급하고 있어 스포츠 주제가 가장 유력합니다.
2. 세계: 월드컵과 한국전이 언급되어 있어 국제적인 스포츠 이벤트로 보입니다.
3. 생활문화: '천재'라는 표현이 사용되어 감독의 능력


In [87]:
# LGAI-EXAONE/EXAONE-3.0-7.8B-Instruct
# 대한항공 우리카드 꺾고 3연승…GS칼텍스 1라운드 전승종합 / 스포츠
# 네이버 모바일 연예판에도 AI 콘텐츠 추천 시스템 적용 / 페이스북 인터넷 드론 아퀼라 실물 첫 시험비행 성공 / IT과학
# NH투자 1월 옵션 만기일 매도 우세 / 금융시장 충격 일단 소강국면…주가 낙폭 줄고 환율도 하락 / 경제
# 황총리 각 부처 비상대비태세 철저히 강구해야 / 정치
# 월미도 새 모노레일 내년 추석엔 달릴 수 있을까 / 사회
# 트럼프 한국 등 방위비분담금 더 내라…양방향 도로 돼야 / 세계 
# 벚꽃 와인 마시며 봄 즐기세요 / 생활문화

# few_text = ["황총리 각 부처 비상대비태세 철저히 강구해야", "금융시장 충격 일단 소강국면…주가 낙폭 줄고 환율도 하락", "월미도 새 모노레일 내년 추석엔 달릴 수 있을까", "벚꽃 와인 마시며 봄 즐기세요", "트럼프 한국 등 방위비분담금 더 내라…양방향 도로 돼야", "네이버 모바일 연예판에도 AI 콘텐츠 추천 시스템 적용", "대한항공 우리카드 꺾고 3연승…GS칼텍스 1라운드 전승종합"]
# few_topic = ["정치", "경제", "사회", "생활문화", "세계", "IT과학", "스포츠"]

In [88]:
# few_shop_promt = ''
# for text, topic in zip(few_text, few_topic):
#     few_shop_promt = few_shop_promt + EXAONE_FEW_SHOT_TEMP.format(text, topic)

In [89]:
# few_shop_promt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n황총리 각 부처 비상대비태세 철저히 강구해야\n[|assistant|]정치[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n금융시장 충격 일단 소강국면…주가 낙폭 줄고 환율도 하락\n[|assistant|]경제[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n월미도 새 모노레일 내년 추석엔 달릴 수 있을까\n[|assistant|]사회[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n벚꽃 와인 마시며 봄 즐기세요\n[|assistant|]생활문화[|endofturn|][|system|] You are EXAONE model from

In [90]:
# EXAONE_TEMP = '''[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
# [|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
# {}
# [|assistant|]'''

In [126]:
# test_text = '코로나 r대^등교)모습'
# test_prompt = EXAONE_TEMP.format(test_text)
# test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n코로나 r대^등교)모습\n[|assistant|]'

In [127]:
# few_shop_promt + test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n황총리 각 부처 비상대비태세 철저히 강구해야\n[|assistant|]정치[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n금융시장 충격 일단 소강국면…주가 낙폭 줄고 환율도 하락\n[|assistant|]경제[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n월미도 새 모노레일 내년 추석엔 달릴 수 있을까\n[|assistant|]사회[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n벚꽃 와인 마시며 봄 즐기세요\n[|assistant|]생활문화[|endofturn|][|system|] You are EXAONE model from

In [128]:
# input_ids = tokenizer(few_shop_promt + test_prompt, return_tensors="pt")['input_ids']
# output = model.generate(
#     input_ids.to("cuda"),
#     eos_token_id=tokenizer.eos_token_id,
#     max_new_tokens=128
# )
# print(tokenizer.decode(output[0]))

In [130]:
# pred = tokenizer.decode(output[0])

In [132]:
# pred.split("[|assistant|]")[-1]

'생활문화\n\n이 헤드라인은 코로나19 팬데믹 상황에서 학생들의 등교 모습을 다루고 있는 것으로 보입니다. 이는 교육과 일상 생활의 변화를 주제로 한 기사로, 생활문화 카테고리에 속합니다.[|endofturn|]'

In [107]:
# test_text = '아시안게임 목소리 높인 박항서 베트남이 일본 못 이길...'
# test_prompt = EXAONE_TEMP.format(test_text)
# test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n아시안게임 목소리 높인 박항서 베트남이 일본 못 이길...\n[|assistant|]'

In [108]:
# few_shop_promt + test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n황총리 각 부처 비상대비태세 철저히 강구해야\n[|assistant|]정치[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n금융시장 충격 일단 소강국면…주가 낙폭 줄고 환율도 하락\n[|assistant|]경제[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n월미도 새 모노레일 내년 추석엔 달릴 수 있을까\n[|assistant|]사회[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n벚꽃 와인 마시며 봄 즐기세요\n[|assistant|]생활문화[|endofturn|][|system|] You are EXAONE model from

In [109]:
# input_ids = tokenizer(few_shop_promt + test_prompt, return_tensors="pt")['input_ids']

In [110]:
output = model.generate(
#     input_ids.to("cuda"),
#     eos_token_id=tokenizer.eos_token_id,
#     max_new_tokens=1
# )
# print(tokenizer.decode(output[0]))

[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
황총리 각 부처 비상대비태세 철저히 강구해야
[|assistant|]정치[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
금융시장 충격 일단 소강국면...주가 낙폭 줄고 환율도 하락
[|assistant|]경제[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
월미도 새 모노레일 내년 추석엔 달릴 수 있을까
[|assistant|]사회[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
벚꽃 와인 마시며 봄 즐기세요
[|assistant|]생활문화[|endofturn|][|system|] You are EXAONE model from LG AI Rese

In [99]:
# test_text = '영상 선출부터 퇴장까지…나경원 원내대표 파란만장 1년'
# test_prompt = EXAONE_TEMP.format(test_text)
# test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n영상 선출부터 퇴장까지…나경원 원내대표 파란만장 1년\n[|assistant|]'

In [100]:
# few_shop_promt + test_prompt

'[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n황총리 각 부처 비상대비태세 철저히 강구해야\n[|assistant|]정치[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n금융시장 충격 일단 소강국면…주가 낙폭 줄고 환율도 하락\n[|assistant|]경제[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n월미도 새 모노레일 내년 추석엔 달릴 수 있을까\n[|assistant|]사회[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]\n[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.\n벚꽃 와인 마시며 봄 즐기세요\n[|assistant|]생활문화[|endofturn|][|system|] You are EXAONE model from

In [101]:
# input_ids = tokenizer(few_shop_promt + test_prompt, return_tensors="pt")['input_ids']

In [105]:
# output = model.generate(
#     input_ids.to("cuda"),
#     eos_token_id=tokenizer.eos_token_id,
#     max_new_tokens=1
# )
# print(tokenizer.decode(output[0]))

[|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
황총리 각 부처 비상대비태세 철저히 강구해야
[|assistant|]정치[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
금융시장 충격 일단 소강국면...주가 낙폭 줄고 환율도 하락
[|assistant|]경제[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
월미도 새 모노레일 내년 추석엔 달릴 수 있을까
[|assistant|]사회[|endofturn|][|system|] You are EXAONE model from LG AI Research, a helpful assistant. [|endofturn|]
[|user|] 다음 뉴스의 헤드라인을 보고 기사가 어떤 주제인제 말해줘. 주제는 정치, 경제, 사회, 생활문화, 세계, IT과학, 스포츠가 있어. 답변은 단답으로 해줘.
벚꽃 와인 마시며 봄 즐기세요
[|assistant|]생활문화[|endofturn|][|system|] You are EXAONE model from LG AI Rese